# Instalamos e importamos librerías

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 63kB/s 
     |████████████████████████████████| 204kB 46.2MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=572d2b4a37e640d67cec291e7a60e082b8151d2d89aaefcd74d105f206308d17
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

# 2020-02 Parcial

Tenemos un RDD con información de recetas:

(ID_Receta, Nombre, Categoría)

Y otro RDD con los ingredientes de cada receta:

(ID_Receta, Ingrediente, Cantidad_Kg)

Queremos obtener:

* a) Listar todos los ingredientes que aparecen en alguna receta que usa "pollo" indicando en
cuantas recetas el ingrediente y pollo aparecen juntos. El formato de salida es (ingrediente,
cantidad de recetas en que aparece junto con pollo). Por ejemplo, la papa aparece en 10
recetas con pollo, por lo que tendríamos (papa, 10). (50 pts)

* b) Queremos obtener todos los nombres de recetas Mediterráneas que no tengan ni papa ni
pollo entre sus ingredientes.(50 pts)

Resolver los puntos usando la API de RDDs de PySpark.

In [ ]:
recetas = [
    (1, 'wok', 'China'),
    (2, 'estofado', 'Italiana'),
    (3, 'tortilla', 'Mediterranea'),
    (4, 'Pollo al horno', 'Mediterranea'),
    (5, 'Ni Pollo ni papa', 'Mediterranea')
]

ingredientes = [
    (1, 'pollo', 0.4),
    (1, 'zanahoria', 0.2),
    (1, 'papa', 0.2),
    (2, 'carne', 1),
    (3, 'papa', 0.5),
    (4, 'pollo', 0.3),
    (4, 'papa', 0.2),
    (4, 'cebolla', 0.1),
    (5, 'carne', 0.1),
    (5, 'sal', 0.01)
]

In [ ]:
recetasRDD = sc.parallelize(recetas)
ingredientesRDD = sc.parallelize(ingredientes)

In [ ]:
recetasPollo = ingredientesRDD.filter(lambda x: x[1] == 'pollo').map(lambda x: (x[0],x[1]))

In [ ]:
recetasPollo.collect()

[(1, 'pollo'), (4, 'pollo')]

In [ ]:
recetasPollo.join(ingredientesRDD.map(lambda x: (x[0],x[1]))).collect()

[(4, ('pollo', 'pollo')),
 (4, ('pollo', 'papa')),
 (4, ('pollo', 'cebolla')),
 (1, ('pollo', 'pollo')),
 (1, ('pollo', 'zanahoria')),
 (1, ('pollo', 'papa'))]

In [ ]:
ingredientesByReceta = ingredientesRDD.map(lambda x: (x[0],x[1])).cache()

In [ ]:
ingredientesConPollo = recetasPollo.join(ingredientesByReceta)\
.filter(lambda x: x[1][1] != 'pollo')

In [ ]:
ingredientesConPollo.collect()

[(4, ('pollo', 'papa')),
 (4, ('pollo', 'cebolla')),
 (1, ('pollo', 'zanahoria')),
 (1, ('pollo', 'papa'))]

In [ ]:
ingredientesCount = ingredientesConPollo.map(lambda x: (x[1][1],1)).reduceByKey(lambda x,y:x+y).cache()

In [ ]:
ingredientesCount.collect()

[('zanahoria', 1), ('papa', 2), ('cebolla', 1)]

In [ ]:
ingredientesCount.filter(lambda x: x[1]>1).collect()

[('papa', 2)]

Punto b

In [1]:
mediterraneas = recetasRDD.filter(lambda x:x[2] == 'Mediterranea').map(lambda x: (x[0],x[1]))

NameError: name 'recetasRDD' is not defined

In [ ]:
mediterraneas.collect()

[(3, 'tortilla'), (4, 'Pollo al horno'), (5, 'Ni Pollo ni papa')]

In [ ]:
mediterraneas.join(ingredientesByReceta).collect()

[(4, ('Pollo al horno', 'pollo')),
 (4, ('Pollo al horno', 'papa')),
 (4, ('Pollo al horno', 'cebolla')),
 (5, ('Ni Pollo ni papa', 'carne')),
 (5, ('Ni Pollo ni papa', 'sal')),
 (3, ('tortilla', 'papa'))]

In [ ]:
mediterraneas.join(ingredientesByReceta)\
.map(lambda x: (x[0],(1 if x[1][1] in ['pollo','papa'] else 0,x[1][0]))).collect()

[(4, (1, 'Pollo al horno')),
 (4, (1, 'Pollo al horno')),
 (4, (0, 'Pollo al horno')),
 (5, (0, 'Ni Pollo ni papa')),
 (5, (0, 'Ni Pollo ni papa')),
 (3, (1, 'tortilla'))]

In [ ]:
mediterraneas.join(ingredientesByReceta)\
.map(lambda x: (x[0],(1 if x[1][1] in ['pollo','papa'] else 0,x[1][0])))\
.reduceByKey(lambda x,y: (x[0]+y[0],x[1])).filter(lambda x: x[1][0] == 0).map(lambda x: x[1][1]).collect()

['Ni Pollo ni papa']

In [ ]:
mediterraneas.join(ingredientesByReceta).map(lambda x: (x[0],1 if x[1][1] in ['pollo','papa'] else 0))\
             .reduceByKey(lambda x,y: x+y).filter(lambda x: x[1] == 0).collect()

[(5, 0)]

In [ ]:
mediterraneas.join(ingredientesByReceta).groupByKey().map(lambda x: (x[0],list(x[1]))).collect()

[(4,
  [('Pollo al horno', 'pollo'),
   ('Pollo al horno', 'papa'),
   ('Pollo al horno', 'cebolla')]),
 (5, [('Ni Pollo ni papa', 'carne'), ('Ni Pollo ni papa', 'sal')]),
 (3, [('tortilla', 'papa')])]